In [1]:
import numpy as np
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(42)
random.seed(42)

from DataExploration import DataExploration
from DecisionTree import DecisionTree
from NeuralNetwork import DNN
from Boosting import Boosting
from KNN import KNN
from NeuralNetworkKeras import NN_Keras
from SVM import SVM

from utils import make_timing_curve

C:\Users\matan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


C:\Users\matan\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


In [2]:
# get datasets ready
de = DataExploration()

twitter_df = de.prepare_twitter_df()
speed_dating_df = de.prepare_speed_dating_df()
encoded_speed_dating_df, labelled_speed_dating_df, label_encoder = de.get_hot_encoded_speed_dating(speed_dating_df)

(22223, 10)


(19276, 10)
(41499, 10)


(8378, 44)
removing empty values shape:
(6794, 44)


In [3]:
X_train, X_test, y_train, y_test = de.get_train_test_validation(encoded_speed_dating_df, 'speed_dating')

In [ ]:
# de.investigate_twitter(twitter_df)
# de.investigate_speed_dating(labelled_speed_dating_df, label_encoder)

# Speed Dating Learners


In [4]:
#best_params = {'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7} #twitter
best_params = {'dt__alpha': 0.0001, 'dt__max_depth': 9, 'dt__min_samples_leaf': 4} #speed_dating  (CV score=0.761):
dt_clf = DecisionTree(model_name='dt_pruning_1', dataset_name='speed_dating', best_params=best_params)
dt_clf.train(X_train, y_train)
y_pred = dt_clf.predict(X_test)
dt_clf.evaluate_model(y_test=y_test, y_pred=y_pred)

#print(dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth)


[[527 107]
 [158 304]]
             precision    recall  f1-score   support

        0.0       0.77      0.83      0.80       634
        1.0       0.74      0.66      0.70       462

avg / total       0.76      0.76      0.76      1096



In [5]:
dt_best_params = dt_clf.get_best_params()
print(dt_best_params)
dt_clf_model = dt_clf.get_model()
#print(dt_clf.feature_importance_)

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, dt_clf_model, 'dt_pruning_1', 'speed_dating',
                              'DT w/ Pruning')


{'dt__alpha': 0.0001, 'dt__max_depth': 9, 'dt__min_samples_leaf': 4}


dt_pruning_1 speed_dating 0.1


dt_pruning_1 speed_dating 0.2


dt_pruning_1 speed_dating 0.3


dt_pruning_1 speed_dating 0.4


...

dt_pruning_1 speed_dating 0.6


dt_pruning_1 speed_dating 0.7


dt_pruning_1 speed_dating 0.8


dt_pruning_1 speed_dating 0.9


        train      test
0.1  0.690082  0.010027
0.2  0.471644  0.006450
0.3  0.395521  0.005813
0.4  0.384189  0.006043
0.5  0.533377  0.006695
0.6  0.457188  0.006273
0.7  0.377044  0.005044
0.8  0.341990  0.005563
0.9  0.481691  0.005762


In [ ]:
# 
# def iteration_lc(clf, X_train, y_train, X_test, y_test, params, clf_type=None, dataset=None,
#                  dataset_readable_name=None, balanced_dataset=False, x_scale='linear', seed=42, threads=1):
#     logger.info("Building iteration learning curve for params {} ({} threads)".format(params, threads))
# 
#     if clf_type is None or dataset is None:
#         raise Exception('clf_type and dataset are required')
#     if seed is not None:
#         np.random.seed(seed)
# 
#     curr_scorer = scorer
#     acc_method = balanced_accuracy
#     if not balanced_dataset:
#         curr_scorer = f1_scorer
#         acc_method = f1_accuracy
# 
#     cv = ms.GridSearchCV(clf, n_jobs=threads, param_grid=params, refit=True, verbose=10, cv=5, scoring=curr_scorer)
#     cv.fit(training_x, training_y)
#     reg_table = pd.DataFrame(cv.cv_results_)
#     reg_table.to_csv('{}/ITER_base_{}_{}.csv'.format(OUTPUT_DIRECTORY, clf_type, dataset), index=False)
#     d = defaultdict(list)
#     name = list(params.keys())[0]
#     for value in list(params.values())[0]:
#         d['param_{}'.format(name)].append(value)
#         clf.set_params(**{name: value})
#         clf.fit(training_x, training_y)
#         pred = clf.predict(training_x)
#         d['train acc'].append(acc_method(training_y, pred))
#         clf.fit(training_x, training_y)
#         pred = clf.predict(test_x)
#         d['test acc'].append(acc_method(test_y, pred))
#         logger.info(' - {}'.format(value))
#     d = pd.DataFrame(d)
#     d.to_csv('{}/ITERtestSET_{}_{}.csv'.format(OUTPUT_DIRECTORY, clf_type, dataset), index=False)
#     plt = plot_learning_curve('{} - {} ({})'.format(clf_type, dataset_readable_name, name),
#                               d['param_{}'.format(name)], d['train acc'], d['test acc'],
#                               multiple_runs=False, x_scale=x_scale,
#                               x_label='Value')
#     plt.savefig('{}/images/{}_{}_ITER_LC.png'.format(OUTPUT_DIRECTORY, clf_type, dataset), format='png', dpi=150)
# 
#     logger.info(" - Iteration learning curve complete")
# 
#     return cv

In [11]:
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}
best_params = {'boost__base_estimator__alpha': 0.01, 'boost__base_estimator__max_depth': 33, 'boost__n_estimators': 40} # (CV score=0.804) speed_dating:
boosting_clf = Boosting(model_name='boosting_1', dataset_name='speed_dating', best_params=best_params)
boosting_clf.train(X_train, y_train)
y_pred = boosting_clf.predict(X_test)
boosting_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


[[496 102]
 [138 360]]
             precision    recall  f1-score   support

        0.0       0.78      0.83      0.81       598
        1.0       0.78      0.72      0.75       498

avg / total       0.78      0.78      0.78      1096



In [13]:
boosting_best_params = boosting_clf.get_best_params()
print(boosting_best_params)
boosting_clf_model = boosting_clf.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, boosting_clf_model, 'boosting_1', 'speed_dating',
                              'Adaboost Classifier')

{'boost__base_estimator__alpha': 0.01, 'boost__base_estimator__max_depth': 33, 'boost__n_estimators': 40}


In [9]:
#best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'} #twitter
best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 121, 'knn__weights': 'distance'} # (CV score=0.749): #speeddating
knn_clf = KNN(model_name='knn_1', dataset_name='speed_dating', best_params=best_params)
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Best parameter (CV score=0.749):
{'knn__metric': 'manhattan', 'knn__n_neighbors': 121, 'knn__weights': 'distance'}


[[562  72]
 [189 273]]
             precision    recall  f1-score   support

        0.0       0.75      0.89      0.81       634
        1.0       0.79      0.59      0.68       462

avg / total       0.77      0.76      0.75      1096



In [10]:
knn_best_params = knn_clf.get_best_params()
print(knn_best_params)
knn_clf_model = knn_clf.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, knn_clf_model, 'knn_1', 'speed_dating',
                              'k-NN')

{'knn__metric': 'manhattan', 'knn__n_neighbors': 121, 'knn__weights': 'distance'}


In [ ]:


#best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
#               'loss': 'mean_squared_error'}
#best_params = {'dnn__batch_size': 100, 'dnn__epochs': 50, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'} #
best_params = {'dnn__batch_size': 100, 'dnn__epochs': 5, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.05, 'dnn__loss': 'binary_crossentropy'} #
keras_nn_clf = NN_Keras(model_name='keras_nn_1', dataset_name='speed_dating', best_params=best_params)
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)



In [14]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)

keras_nn_clf_model = keras_nn_clf.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, keras_nn_clf_model, 'keras_nn_1', 'speed_dating',
                              'Feed-Forward Neural Network')

{'dnn__batch_size': 50, 'dnn__epochs': 30, 'dnn__layer_sizes': [100, 20], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'binary_crossentropy'}


In [15]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
best_params = {'svm__C': 2, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'} # (CV score=0.781)
svm_clf_rbf = SVM(model_name='svm_rbf_1', dataset_name='speed_dating', best_params=best_params)
svm_clf_rbf.train(X_train, y_train,'rbf')
y_pred = svm_clf_rbf.predict(X_test)
svm_clf_rbf.evaluate_model(y_test=y_test, y_pred=y_pred)


KeyboardInterrupt: 

In [16]:
svm_best_params = svm_clf_rbf.get_best_params()
print(svm_best_params) 
svm_clf_rbf_model = svm_clf_rbf.get_model()
#{'svm__degree': 1, 'svm__kernel': 'rbf'}
# (CV score=0.781):
# {'svm__C': 16.0, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'}

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_rbf_model, 'svm_rbf_1', 'speed_dating',
                              'SVM (RBF)')

{'svm__C': 16.0, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'}


In [17]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
best_params = {'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly', 'svm__degree':19} #r (CV score=0.764): 

svm_clf_poly = SVM(model_name='svm_poly_1', dataset_name='speed_dating', best_params=best_params)
svm_clf_poly.train(X_train, y_train,'poly')
y_pred = svm_clf_poly.predict(X_test)
svm_clf_poly.evaluate_model(y_test=y_test, y_pred=y_pred)


[[498 136]
 [131 331]]
             precision    recall  f1-score   support

        0.0       0.79      0.79      0.79       634
        1.0       0.71      0.72      0.71       462

avg / total       0.76      0.76      0.76      1096



In [18]:
svm_best_params = svm_clf_poly.get_best_params()
print(svm_best_params) 
svm_clf_poly_model = svm_clf_poly.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_poly_model, 'svm_poly_1', 'speed_dating',
                              'SVM (Poly)')

{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'}


In [19]:
#Best parameter (CV score=0.784):
best_params = {'svm__C': 0.03125, 'svm__gamma': 0.001953125, 'svm__kernel': 'linear'}

svm_clf_linear_sigmoid = SVM(model_name='svm_linear_sigmoid_1', dataset_name='speed_dating', best_params=best_params)
svm_clf_linear_sigmoid.train(X_train, y_train,'linear_sigmoid')
y_pred = svm_clf_linear_sigmoid.predict(X_test)
svm_clf_linear_sigmoid.evaluate_model(y_test=y_test, y_pred=y_pred)


In [20]:
svm_best_params = svm_clf_linear_sigmoid.get_best_params()
print(svm_best_params) 
svm_clf_linear_sigmoid_model = svm_clf_linear_sigmoid.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_poly_model, 'svm_sigmoid_linear_1', 'speed_dating',
                              'SVM (Linear)')

# Twitter Learners

In [21]:
X_train, X_test, y_train, y_test, = de.get_train_test_validation(twitter_df, 'twitter')


ValueError: not enough values to unpack (expected 8, got 4)

In [22]:
best_params = {'dt__alpha': 0.0001, 'dt__max_depth': 17, 'dt__min_samples_leaf': 9} #twitter(CV score=0.912)
dt_clf_t = DecisionTree(model_name='dt_pruning_1', dataset_name='twitter', best_params=best_params)
dt_clf_t.train(X_train, y_train)
y_pred = dt_clf_t.predict(X_test)
dt_clf_t.evaluate_model(y_test=y_test, y_pred=y_pred)

#print(dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth)


ValueError: Number of weights=3505 does not match number of samples=2804

In [23]:
dt_best_params = dt_clf_t.get_best_params()
print(dt_best_params)
dt_clf_t_model = dt_clf_t.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, dt_clf_t_model, 'dt_pruning_1', 'twitter',
                              'DT w/ Pruning')


None


In [24]:
best_params = {'boost__base_estimator__alpha': 0.01, 'boost__base_estimator__max_depth': 25, 'boost__n_estimators': 40} #(CV score=0.918):
boosting_clf_t = Boosting(model_name='boosting_1', dataset_name='twitter', best_params=best_params)
boosting_clf_t.train(X_train, y_train)
y_pred = boosting_clf_t.predict(X_test)
boosting_clf_t.evaluate_model(y_test=y_test, y_pred=y_pred)


ValueError: Number of weights=3505 does not match number of samples=2804

In [25]:
boosting_best_params = boosting_clf_t.get_best_params()
print(boosting_best_params)
boosting_clf_t_model = boosting_clf.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, boosting_clf_t_model, 'boosting_1', 'twitter',
                              'Adaboost Classifier')


None


In [26]:
best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'} #twitter
#{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'} (CV score=0.728): #speeddating
knn_clf_t = KNN(model_name='knn_1', dataset_name='twitter', best_params=best_params)
knn_clf_t.train(X_train, y_train)
y_pred = knn_clf_t.predict(X_test)
knn_clf_t.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
knn_best_params = knn_clf_t.get_best_params()
print(knn_best_params)
knn_clf_t_model = knn_clf_t.get_model()


timing_df = make_timing_curve(X_train, y_train, X_test, y_test, knn_clf_t_model, 'knn_1', 'twitter',
                              'k-NN')

In [ ]:
best_params = {'svm__C': 2.0, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
svm_clf_rbf_t = SVM(model_name='svm_rbf_1', dataset_name='twitter', best_params=best_params)
svm_clf_rbf_t.train(X_train, y_train,'rbf')
y_pred = svm_clf_rbf_t.predict(X_test)
svm_clf_rbf_t.evaluate_model(y_test=y_test, y_pred=y_pred)

In [ ]:
svm_best_params = svm_clf_rbf_t.get_best_params()
print(svm_best_params) 
svm_clf_rbf_t_model = svm_clf_rbf_t.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_rbf_t_model, 'svm_rbf_1', 'twitter',
                              'SVM (RBF)')
#Best parameter (CV score=0.893):
#{'svm__C': 16.0, 'svm__gamma': 1.0, 'svm__kernel': 'rbf'}

In [ ]:
# #best_params = 
# #{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):
# 
# svm_clf_poly_t = SVM(model_name='svm_poly_1', dataset_name='twitter')
# svm_clf_poly_t.train(X_train, y_train,'poly')
# y_pred = svm_clf_poly_t.predict(X_test)
# svm_clf_poly_t.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
# svm_best_params = svm_clf_poly.get_best_params()
# print(svm_best_params) 
# svm_clf_poly_t_model = svm_clf_poly_t.get_model()
# #{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [ ]:
# #best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
# #{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):
# 
# svm_clf_linear_sigmoid_t = SVM(model_name='svm_wip_linear_sigmoid', dataset_name='twitter')
# svm_clf_linear_sigmoid_t.train(X_train, y_train,'linear_sigmoid')
# y_pred = svm_clf_linear_sigmoid_t.predict(X_test)
# svm_clf_linear_sigmoid_t.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
# svm_best_params = svm_clf_linear_sigmoid_t.get_best_params()
# print(svm_best_params) 
# svm_clf_linear_sigmoid_t_model = svm_clf_linear_sigmoid_t.get_model()
# 
# timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_linear_sigmoid_t_model, 'svm_sigmoid_linear_1', 'twitter',
#                               'SVM (Linear)')
# #{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [ ]:
best_params = est_params = {'dnn__batch_size': 100, 'dnn__epochs': 5, 'dnn__layer_sizes': [12, 5], 'dnn__learning_rate': 0.05, 'dnn__loss': 'binary_crossentropy'} #
#{'dnn__batch_size': 10, 'dnn__epochs': 1, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'} #
keras_nn_clf = NN_Keras(model_name='keras_nn_1', dataset_name='twitter')
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)
keras_nn_clf_model_t = keras_nn_clf.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, keras_nn_clf_model_t, 'keras_nn_1', 'twitter',
                              'Feed-Forward Neural Network')